# CS1P Lab exam 2021/2022

## Rules

* **Read these rules very carefully before you begin.**

<div class="alert alert-danger">
    
This submission must be 100% your own work. Do not discuss, share or collaborate on any part of this exam with any other person.
    
</div>

1. This is an **open book** exam. You **may** consult external references (books, Stack Overflow, etc.) but you **must not** copy and paste code verbatim; nor may you reveal or discuss any aspect of the exam with anyone in an online (or offline) medium.
1. You must submit the correct file on Moodle by the posted deadline. **There are no extensions.** If you submit the wrong thing, you will get 0 marks.
1. **You may not import *any* libraries beyond those already imported for you.** You do not need to use a library just because it is imported! If you import a library, you will lose two bands automatically, and receive no credit for answers that rely on that library.

---

### Marking
1. This exam is marked out of 75.
1. The division of marks is listed by each task.
1. The parts increase in difficulty. Remember: partial solutions will get credit. You do not have to have completely working code, or implement all of the features requested, to get most of the marks. Make an attempt if you can.
1. **You should expect that you will not be able to complete all parts of this exam unless you are very fluent with the course material.**
1. You are warned that spending excess time will not likely increase your grade but will increase your stress levels. 
1. You do not need to comment your code in this exam.
   

## Important
Please enter, into the cell marked **[STATEMENT HERE]**, the following statement:

> I, [your name], have read and understand the rules governing this lab exam and I will abide by them.
    
(double-click a cell to edit it).


I, Miko Osak, have read and understand the rules governing this lab exam and I will abide by them.

---

# Problem: A simple blockchain

In [1]:
# these are the only imports permitted
import hashlib, secrets

## Background
A **blockchain** is a way of storing verifiable **records** without relying on a central party like a bank. It is the underlying principle behind cryptocurrencies like Bitcoin or Ethereum, as well as having other uses. We can assume we represent each **record** as a text string.

A record held in a blockchain might be used to hold something like a *ledger*, which is just a list of transactions indicating which people received money from other people.

An example ledger might be:

    Bob => Alice £200 2021-11-08-12-09-03
    Alice => Charlie £30  2021-11-08-14-29-33
    Andy => Bob £10  2021-11-08-18-10-01
    
    
which is in the format

    [sender] => [receiver] £[amount] [datetime]
    
`datetime` is in the format `yyyy-mm-dd-hh-MM-ss` (year, month, day, hours, minutes, seconds).
    

## Part 1: Reading ledgers [30 marks]
In this part you will be marked on correctness of your code. Tests are provided for you. Passing the tests does *not* mean your code will get full marks, but failing the tests will mean you cannot get full marks. You are *not* asked to write additional tests.

### Task 1.(a) Read ledgers

Write a function `read_ledger(s)` that takes a string formatted as above and returns a list of transactions, as a list of tuples `[(sender, receiver, amount, datetime), ...]`, all of which should be strings, except `amount`, which should be an integer (**not** a float). Assume `s` is always correctly formatted; you don't need to check for errors.

**[8 marks]**

In [2]:
def read_ledger(s):
    count = 0
    temp_list = []
    output = []
    
    ledger_str = s.replace("\n", " ").split(" ") # ledger into string
    
    for i in range(3,len(ledger_str),5): # convert money value to int
        ledger_str[i] = int(ledger_str[i][1:])
        
    for index, item in enumerate(ledger_str):
        if item == "=>": # ignore the arrow sign
            continue
        else:
            temp_list.append(item) # put into temp list otherwise
            
        if (index + 1) % 5 == 0: # when we reach datetime
            output.append(tuple(temp_list)) # append a tuple of the temp list into output
            temp_list = [] # reset temp list
                   
    return output

In [3]:
## Tests

ledger = read_ledger("""Bob => Alice £200 2021-11-08-12-09-03
Alice => Charlie £30 2021-11-08-14-29-33
Andy => Bob £10 2021-11-08-18-10-01""")

assert len(ledger)==3
assert ledger[0] == ("Bob", "Alice", 200, "2021-11-08-12-09-03")
assert ledger[-1][-2] == 10
assert type(ledger[-1][-2])==int

ledger = read_ledger("")

assert len(ledger)==0

ledger = read_ledger("Andy => Bob £10 2021-11-08-18-10-01")
assert len(ledger)==1

### Task 1.(b) Compute totals 

Write a function `totalise_ledger(ledger)` which will take a list of tuples as returned from `read_ledger` and will return a dictionary mapping each person in the ledger to their final balance after all transactions. To do this, just add up each amount of money going to and from a person. For example, the ledger example above would result in:

    {"Bob":-190, "Alice":170, "Charlie":30, "Andy":-10}
    
**[8 marks]**

In [4]:
def totalise_ledger(ledger):
    output = {}
    
    for tup in ledger: # make key for every name in ledger
        if output.get(tup[0], None) is None:
            output[tup[0]] = 0
        if output.get(tup[1], None) is None:
            output[tup[1]] = 0
            
    for tup in ledger: # change dict value for every transaction
        amount = tup[2]
        output[tup[0]] = output[tup[0]] - amount
        output[tup[1]] = output[tup[1]] + amount
            
    return output

In [5]:
## Tests

ledger = [('Bob', 'Alice', 200, '2021-11-08-12-09-03'),
 ('Alice', 'Charlie', 30, '2021-11-08-14-29-33'),
 ('Andy', 'Bob', 10, '2021-11-08-18-10-01')]

totals = totalise_ledger(ledger)

assert type(totals)==type({})
assert len(totals)==4
assert totals["Bob"]==-190
assert totals["Alice"]==170

                          
ledger = [('Bob', 'Alice', 0, '2021-11-08-12-09-03')]

totals = totalise_ledger(ledger)
assert totals["Bob"]==0
assert totals["Alice"]==0
    

assert len(totalise_ledger(read_ledger("")))==0

### Task 1.(c) Validate timestamp orders

Ledgers need to be in the right time order, with each transaction having a timestamp after the previous entry. A ledger with transactions out of time order is invalid. For example, this ledger is invalid, as the timestamps are not in order:

    Bob => Alice £20 2021-12-08-08-19-29
    Bob => Charlie £10 2021-11-08-18-10-01
    Bob => Charlie £10 2021-11-08-19-22-51

Write a function `validate_ledger(s)` that will return `True` if a ledger -- a list of any number of ledgers in tuple format, as `read_ledger()` would return -- is in valid order, and `False` otherwise.

**[6 marks]**

In [6]:
def validate_ledger(s):
    current_datetime = []
    nex_t_datetime = []
    valid = False
    
    if len(s) == 0: # need to take into account a list with zero ledgers in tuple format
        return True
    
    for index, tup in enumerate(s): # avoid going out of the index
        if index + 1 >= len(s):
            break
        
        current_datetime = tup[3].split("-") # get datetime as list
        nex_t_datetime = s[index + 1][3].split("-") # get the datetime to compare it as list
        
        for time_index, time in enumerate(current_datetime): # time to start comparing every timestamp
            
            if current_datetime[time_index] < nex_t_datetime[time_index]: # if the time is less than the time it's comparing to
                valid = True # then is has to be valid because of the order of the times
                break # so there is no need to check these times further
                
            if current_datetime[time_index] > nex_t_datetime[time_index]: # same as above but reversed
                valid = False
                break
        
    return valid

In [7]:
## Tests

assert validate_ledger([('Bob', 'Alice', 200, '2021-11-08-12-09-03'),
 ('Alice', 'Charlie', 30, '2021-11-08-14-29-33'),
 ('Andy', 'Bob', 10, '2021-11-08-18-10-01')])

assert validate_ledger([])
                                                                        
assert not validate_ledger([('Bob', 'Alice', 20, '2021-12-08-08-19-29'),
 ('Bob', 'Charlie', 10, '2021-11-08-18-10-01')])

### Task 1.(d) Merge ledgers

Two valid (i.e. in order) ledgers can be merged. To do this, we need to fold together all transactions in order. For example, if we had the two ledgers:


    Bob => Alice £200 2021-11-08-12-09-03
    Alice => Charlie £30  2021-11-08-14-29-33
    Andy => Bob £10  2021-11-08-18-10-01
    
    and    
    
    Bob => Andy £10 2021-11-08-04-13-23
    Farah => Alice £70 2021-11-08-15-13-23
    Susan => Bob £100 2021-11-15-09-55-01
    
then the merged version would be:

    Bob => Andy £10 2021-11-08-04-13-23
    Bob => Alice £200 2021-11-08-12-09-03
    Alice => Charlie £30  2021-11-08-14-29-33
    Farah => Alice £70 2021-11-08-15-13-23
    Andy => Bob £10  2021-11-08-18-10-01
    Susan => Bob £100 2021-11-15-09-55-01

Write:

* a function `merge_two_ledgers(ledger_a, ledger_b)` which takes a pair of valid ledgers and merges them into one valid ledger and returns it.

* Then, write a function `merge_ledgers(ledgers)` which uses `merge_two_ledgers` and takes a list of *any number* of valid ledgers and then merges them into one single ledger and returns it.  

Assume you are always passed valid (i.e. time-ordered ledgers). You do not need to consider the case of equal timestamps; assume all timestamps are unique.

**Solve this without using sorting; if you use sorting (`.sort`, `sorted()`, or your own algorithm), you will receive a maximum of 6 marks for this question.**    

**[8 marks]**

In [8]:
def merge_two_ledgers(ledger_a, ledger_b): # don't know how to sort without using sort, thus the order matters
    if len(ledger_a) >= len(ledger_b): # output length depends on longest input
        count = len(ledger_a)
    if len(ledger_b) > len(ledger_a):
        count = len(ledger_b)
        
    output = []

    for i in range(count):
        if i >= len(ledger_a): # avoid out of index errors
            pass
        else:
            output.append(ledger_a[i]) # else put in value from a

        if i >= len(ledger_b):
            pass
        else:
            output.append(ledger_b[i]) # then put in value from b next to a

    return output

def merge_ledgers(ledgers):
    
    if len(ledgers) == 1: # if we have one ledger, it means that we have our desired result
        return ledgers[0]
    
    elif len(ledgers) == 0: # spit out empty inputs
        return ledgers
    
    else:
        if len(ledgers) == 2: # different returns are needed depending on length
            return merge_ledgers([merge_two_ledgers(ledgers[0], ledgers[1])])
        else:
            return merge_ledgers([merge_two_ledgers(ledgers[0], ledgers[1]), ledgers[2:]]) # this make infinite empty lists if you give if a length 2, thus the if statement fixes that

In [9]:
ledger_1 = [('Bob', 'Alice', 200, '2021-11-08-12-09-03'),
 ('Alice', 'Charlie', 30, '2021-11-08-14-29-33'),
 ('Andy', 'Bob', 10, '2021-11-08-18-10-01')]
    
ledger_2=[('Bob', 'Andy', 10, '2021-11-08-04-13-23'),
 ('Farah', 'Alice', 70, '2021-11-08-15-13-23'),
 ('Susan', 'Bob', 100, '2021-11-15-09-55-01')]

ledger_3=[('Ali', 'Dave', 20, '2020-11-08-04-13-23'),
 ('Mohamed', 'Bob', 20, '2022-11-15-09-55-01')]

merged = merge_two_ledgers(ledger_1, [])
assert merged==ledger_1

merged = merge_two_ledgers([], ledger_2)
assert merged==ledger_2

merged = merge_two_ledgers(ledger_1, ledger_2)
assert [l[0] for l in merged] == ["Bob", "Bob", "Alice", "Farah", "Andy", "Susan"]
assert len(merged)==6

assert merge_two_ledgers(ledger_1, ledger_2)==merge_two_ledgers(ledger_2, ledger_1) # FAILED, don't know how to sort

merged = merge_two_ledgers([], [])
assert merged==[]

merged = merge_ledgers([ledger_1, ledger_2])

assert len(merged)==6
assert [l[0] for l in merged] == ["Bob", "Bob", "Alice", "Farah", "Andy", "Susan"]


merged = merge_ledgers([ledger_2, ledger_1])

assert len(merged)==6
assert [l[0] for l in merged] == ["Bob", "Bob", "Alice", "Farah", "Andy", "Susan"] # FAILED, don't know how to sort 


merged = merge_ledgers([ledger_2, ledger_1, ledger_3])

assert len(merged)==8
assert [l[0] for l in merged] == ["Ali", "Bob", "Bob", "Alice", "Farah", "Andy", "Susan", "Mohamed"] # FAILED, don't know how to sort 


merged = merge_ledgers([ledger_3, [], ledger_2, ledger_1, []])

assert len(merged)==8
assert [l[0] for l in merged] == ["Ali", "Bob", "Bob", "Alice", "Farah", "Andy", "Susan", "Mohamed"] # FAILED, don't know how to sort


merged = merge_ledgers([ledger_1, []])

assert merged==ledger_1

merged = merge_ledgers([])
assert merged==[]

merged = merge_ledgers([ledger_1])
assert merged==ledger_1


AssertionError: 

# Part 2: The blockchain [45 marks]

## Background
The theory of a basic blockchain is simple. 
 
* We have a function `secure_hash(s)` which takes some string and returns a **secure hash** -- a number that is a condensed summary of the string
    * This is known to be very hard to "reverse" to recover the original string, and essentially impossible to modify the string without completely changing the hash.
* We have a sequence of records (called "blocks"), where each block includes the hash of the previous record as well as the current record (the "chain links"). 
* In this way, we can tell if any block in the chain has been modified, because the hashes won't match up.
    

We can use this to ensure that a chain of blocks have not been interfered with by a malicious party, because each block guarantees the integrity of the previous block.


### Block format

The format of a blockchain **block** (i.e. one record in the sequence of records) that we will use is a simple string, which we imagine as divided into parts as follows:

    [random_id][prev_hash][payload_text][hash]

where:
* `random_id` is 64 random hexadecimal digits (assumed to be unique)
* `prev_hash` is the hash copied from the previous entry in the chain (also 64 hex digits)
* `payload_text` is the text we want to store, of any length, stored as plain text.
* `hash` is the secure hash of the `random_id`, `prev_hash` and `text` joined together (in that order), as a 64 digit hex string.   


Two functions and a constant are defined below:

In [10]:
HEXBLOCK = 64

def secure_hash(text):
    """Return the hash of a string, as
    64 hex digit string"""
    h = hashlib.sha256()
    h.update(text.encode("utf-8"))
    return h.hexdigest()

def random_hex(n):
    """Return a random hexadecimal string, which
    has n hexadecimal digits"""
    return secrets.token_hex(n//2+2)[:n]

In [11]:
# returns a string
print(secure_hash("hello"))

# any tiny change completely changes the hash
print(secure_hash("hell0"))

2cf24dba5fb0a30e26e83b2ac5b9e29e1b161e5c1fa7425e73043362938b9824
bdeddd433637173928fe7202b663157c9e1881c3e4da1d45e8fff8fb944a4868


In [12]:
# print some random data in hex
print(random_hex(HEXBLOCK))

a4d8398798b552fa8f5be8d58edeb72ae5b4f91d6561828a9345f9350fd1d4c8


### Task 2(a) Construct the chain

Define the following functions:

* `chain(payload_text, prev)` that takes a previous block in the blockchain and a new `payload_text`, and returns the next block in the blockchain. If `prev` is not given, a so called **root block** should be returned, which has `prev_hash` fixed to all zeros; this would be used for the *first* block in a blockchain. **[8 marks]**

* `append_chain(blocks, text)` which takes an entire blockchain (as a list of strings), and appends a new block generated using a specified payload text, using `chain()`. **[4 marks]**

* `make_chain(texts)` which takes a list of payload texts, and returns a chain containing each payload text as a list of blocks. **[4 marks]**

* `decode_chain(blocks)` which takes a list of blocks and returns the list of corresponding payload texts. **[4 marks]**

Your code should work such that:

    decode_chain(make_chain(texts)) == texts
    
for any list of strings `texts`.

Make sure you re-use functions you have defined. You *do not* have to define any tests, but none are provided.


In [13]:
def chain(payload_text, prev = (random_hex(HEXBLOCK))):
    return secure_hash("".join([random_hex(64), prev, payload_text]))

def append_chain(blocks, text):
    if len(blocks) == 0: # if chain is empty, make root block
        blocks.append(chain(text))
    else: # else make block based off previous block
        blocks.append(chain(text, blocks[-1]))
    return blocks

def make_chain(texts, output = []):
    if len(texts) == 0: # base case, no texts are left to add
        return output
    else:
        if len(output) == 0: # make root block if chain is empty
            output.append(chain(texts[0]))
        else: # or append new block
            output = append_chain(output, texts[0])
    return make_chain(texts[1:], output) # recur with next text and new output

def decode_chain(blocks): # No idea how to decode sha256
    return

example_chain = make_chain(["Hello,", "I'm", "quite"])
print(example_chain)
print()
print(append_chain(example_chain, "hungry."))              

['a8be25e206046df6849cfd44d49fbb3de01438c46c5ead6ce8f73655071e28e8', '1140c7bf215e9077fb5193fa399c3bb50d7f09e24a9bdc0b357dd615e76a9e7f', 'b4f1ee3b432cef761151231f17a8fd4469b7694638beff2aa2278f54a10b7469']

['a8be25e206046df6849cfd44d49fbb3de01438c46c5ead6ce8f73655071e28e8', '1140c7bf215e9077fb5193fa399c3bb50d7f09e24a9bdc0b357dd615e76a9e7f', 'b4f1ee3b432cef761151231f17a8fd4469b7694638beff2aa2278f54a10b7469', 'e33488ea81c23c66ad850aa233e4f2b09b7c1101254c497faa2b7a853fe00004']


### Task 2(b) Validate the chain

Write the function:

* `validate_chain(blocks)` which takes a list of blocks and returns `True` if:
    * every block has a `random_id`, `prev_hash` and `hash` of the correct format (64 hex digits)
    * the `hash` matches the `secure_hash` of the `random_id`, `payload` and the `prev_hash`, as defined above.
         * NOTE: It must not be possible for someone to "forge" a block by creating a block with a mismatched hash -- you must check for this.
    * the sequence of blocks starts at a root block.
    * if any of these conditions are not met, the function should return `False`.
    * **In the case of an invalid chain, the function should print out the index of the block that failed, and the reason for the failure before returning.**
    
Use functions to break up your code.
              

**[15 marks]**

In [14]:
def validate_type(block, index):
    if type(block) != str:
        print("Block", index, "is invalid. Type is", type(block), "instead of string.")
        return False
    return True

def validate_length(block, index):
    if len(block) != 64:
        print("Block", index, "is invalid. Length is", len(block), "instead of 64.")
        return False
    return True

def validate_hex(block, index):
    for char in block:
        if ord(char) < 48 or ord(char) > 57:
            if ord(char) < 97 or ord(char) > 102:
                print("Block", index, "is invalid.", char, "is not a hexadecimal character.")
                return False
    return True

def validate_chain(blocks):
    if len(blocks) == 0: # accounts for empty lists, strings, etc
        print("The given chain is invalid. It is empty, no index can be given.")
        return False
    
    for index, block in enumerate(blocks):
        if validate_type(block, index) == False:
            return False
        if validate_length(block, index) == False:
            return False
        if validate_hex(block, index) == False:
            return False
    return True

# No hash validation as I don't know how to decode hash values.

### Task 2(c): Test
Write tests to check that `validate_chain` works correctly. **[10 marks]**

In [15]:
assert validate_chain([1111111111111111111111111111111111111111111111111111111111111111]) == False # int type invalid

assert validate_chain(["111111111111111111111111111111111111111111111111111111111111111"]) == False # length invalid 63/64

assert validate_chain(["111111411111111111111711111001111111110911af111111111111111111gg"]) == False # hex invalid

assert validate_chain(["1111111111111111111111111111111111111111111111111111111111111111", "1" * 64, ["123"]]) == False # list invalid

assert validate_chain(['bbddce3fd965489aa9b72eb7343c3fcb79b4e1263daca130cdb4658087ce855a'
                       , 'f4a3849992a1302463b688b574e5ef91e02e64bc0bb54a9ed3518932008aad1a'
                       , 'b06d09066e90f87927ef94ca3fd82abb9c40b55dd8a6c700f1dff78d088f9d66'
                       , 'eb4b220d196ccecd50290d56d40cca1d4d0c97b2947720910898bc16840838o9']) == False # hex invalid (2nd last character)

assert validate_chain(["1" * 64, "2" * 64, "3" * 64, float(1 * (10 ** 63))]) == False # float invalid

assert validate_chain([]) == False # empty invalid

assert validate_chain([[]]) == False # list type invalid

assert validate_chain("") == False # empty invalid

assert validate_chain(example_chain) # test example from task 2a, it passes

assert validate_chain(make_chain(["{i}".format(i = i) for i in range(100)])) # big fresh chain, passes

# Naturally my testing is limited as I can't compare hashes.

Block 0 is invalid. Type is <class 'int'> instead of string.
Block 0 is invalid. Length is 63 instead of 64.
Block 0 is invalid. g is not a hexadecimal character.
Block 2 is invalid. Type is <class 'list'> instead of string.
Block 3 is invalid. o is not a hexadecimal character.
Block 3 is invalid. Type is <class 'float'> instead of string.
The given chain is invalid. It is empty, no index can be given.
Block 0 is invalid. Type is <class 'list'> instead of string.
The given chain is invalid. It is empty, no index can be given.



# END OF EXAM

Please:

* Take a break.
* Make sure you read each question carefully. The number one reason to lose marks is to not read the question!
* Check that each of your cells run as you expect. Try `Kernel/Restart and Run All` to make sure.
* Submit your solution on Moodle 
* And then relax :) 

---



## Epilogue

<div class="alert alert-info">
    
This section is not part of the exam tasks. You **do not** have to read this to complete the exam.
</div>

## Problems with blockchains
We've implemented (part) of a very simple blockchain. You will hear a lot about blockchains, crypto, NFTs and related concepts. There are issues with these technologies you should understand. I've provided this summary as it is important to put these ideas in context; however, I would urge you to do your own background research and think critically about what is said.

Blockchains and cryptocurrencies are not synonymous, though cryptocurrencies are the most prominent use of blockchains. Blockchains underpin other processes; version control systems like `git` are akin to blockchains. All of these are ultimately based on the idea of a [Merkle tree](https://en.wikipedia.org/wiki/Merkle_tree), which is the data structure that you have implemented.

### Security and technical

We have only implemented part of a blockchain, and a very simple one at that. Importantly, we have not addressed ways of proving ownership, and (in the context of cryptocurrency) solving the problem of being able to create "money from nothing" (or the "double spending problem"). This would usually involve:

* **Cryptographic signatures** to sign blocks, so that someone can prove that they are the one who made an entry (and thus prove ownership).
* A **consensus mechanism** to allow a consistent blockchain history to be constructed without the involvement a central authority.
* **Proof-of-work** which is used to make it hard to fake blocks by requiring (enormous) computational effort to add new blocks to the chain. Other proof schemes exist, but most current cryptocurrencies primarily use proof-of-work.

If you want to understand how a full cryptocurrency works in more detail, watch [the 3blue1brown video on the topic](https://www.youtube.com/watch?v=bBC-nXj3Ng4)

### Environmental
Because many actively-used blockchains (e.g. in cryptocurrencies) use proof-of-work, there is a huge computational burden to making transactions on a blockchain. The proof-of-work requires truly mind-boggling amounts of electricity to do these computations (not to mention GPUs and other computer hardware), all of which *do nothing* -- they literally prove that someone has wasted enough computation time. 

Bitcoin transactions  [consume more electrical power than the whole of Finland at the time of writing](https://www.nytimes.com/interactive/2021/09/03/climate/bitcoin-carbon-footprint-electricity.html#:~:text=But%20first%2C%20consider%20this%3A%20The,nation%20of%20about%205.5%20million.). This has an extraordinary cost in terms of environmental impact, as it typically requires burning fossil fuels to do the (wasted) computations to prove ownership. It is possible that such schemes will be regulated out of existence by states (e.g. Bitcoin is banned in China and several other states; Sweden is proposing an EU-wide ban). 

Other blockchains may be more environmentally friendly, but they have not yet become dominant.

## Ethical

### Scams
**Most cryptocurrencies are either outright scams or are a front for money laundering.** There are, or could be, legitimate uses for financial transactions based on blockchains, and there is clearly appetite for decentralised systems to move funds. But most people promoting "crypto" are trying to get rich quick, either by speculating (legal but risky), by undermining the rule of law by trading contraband or trading in illegal ways (e.g. selling drugs, evading taxes), or by organising pump and dump schemes (a standard, and illegal, con). There is no free money. There is a lot of money in tricking stupid people.

Many cryptocurrencies, like Bitcoin, have an inherently deflationary structure -- it gets harder to "mine" (do proof-of-work to validate transactions) as time goes on, so it is easy to build up ownership of "coins" at the launch of a cryptocurrency than later. This rewards early adopters in a way that is reminiscent of a pyramid scheme, and occasionally is explicitly a pyramid scheme. Even if not, deflationary currencies incentivise investors to hold onto their investments and not spend it on economic activity; a potentially problematic structure for efficient economies.

### NFTs
Non-fungible tokens (NFTs), which are used to confer a form of "ownership" over artworks have become very popular. These essential issue "receipts" to "owners" and store the receipts on the blockchain, so that someone can prove they "own" an artwork. No actual legal change of ownership of the artwork (e.g. copyright) is usually involved. These are extremely controversial. They have provided much-needed income to artists who otherwise have few ways of monetising digital content. However, many NFT schemes are a transparent con game. No real ownership exists or is transferred -- just a link to a database that belongs to a third party company. Prices are artificially inflated by illegal market manipulation. If someone tells you the future is NFTs, or `web3`, treat them with great scepticism.

### Irrevocability 
Blockchains provide a way to establish a distributed, verifiable history. By their nature, they cannot be modified once established; the chain will no longer verify. History on the blockchain is permanent. This can be good (e.g. if you want to record election results that cannot be tampered with, or property transaction records) but also bad (e.g. if you incorrectly record a criminal conviction). Various legal frameworks, such as the GDPR, require that individuals be able to permanently delete, or correct, data recorded about them. Most blockchains inherently cannot comply with these requirements, and so in many cases identifiable personal data could not legally be stored on a blockchain. 
